In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

from collections import defaultdict, namedtuple


## Handle Taxon Synonymy

In [2]:
# Some taxa need modified search terms. This dict should work for all datasets.
taxa_searchterms = {'Limosilactobacillus fermentum':'Lactobacillus fermentum|Limosilactobacillus fermentum',
                       "Veillonella rogosae": "Veillonella",
                      "Prevotella corporis": "Prevotella",
                      'Bacillus subtilis': 'Bacillus subtilis|Bacillus spizizenii',
                      'Luteovulum sphaeroides':'Rhodobacter sphaeroides|Luteovulum sphaeroides|Cereibacter sphaeroides',
                      'Phocaeicola vulgatus': 'Bacteroides vulgatus|Phocaeicola vulgatus',
                      'Luteovulum':'Rhodobacter|Luteovulum|Cereibacter',
                      'Phocaeicola':'Bacteroides|Phocaeicola',
                      'Limosilactobacillus': 'Lactobacillus|Limosilactobacillus'
                       }

## Name datasets and load Theoretical Distributions

In [42]:
all_datasets = {"ATCC_MSA1003": ["HiFi_ATCC_MSA1003_full", "HiFi_ATCC_MSA1003_short", "Illumina_ATCC_MSA1003"],
              "Zymo_D6300": ["Illumina_Zymo_D6300","ONT_Q20_Zymo_D6300_full","ONT_Q20_Zymo_D6300_short","ONT_R10_Zymo_D6300"],
              "HiFi_Zymo_D6331": ["HiFi_Zymo_D6331"]
}
tools= ["Bracken", "BugSeq-V2", "Centrifuge-h22", 
        "Centrifuge-h500", "Kraken2", "MEGAN-LR-Nuc-HiFi", 
        "MEGAN-LR-Nuc-ONT", "MEGAN-LR-Prot", "MMseqs2", 
        "Metamaps", "Metaphlan3", "mOTUs"]

comparable_lr_tools = ["MEGAN-LR-Nuc-HiFi", "MEGAN-LR-Nuc-ONT", "MEGAN-LR-Prot", "BugSeq-V2", "MMseqs2"]# ,#"Metamaps","Centrifuge-h500"]

In [4]:
# filenames for all theoretical distributions
species_filenames = {"Zymo_D6300": "Zymo_D6300.theoretical-distrib.species.csv",
                     "HiFi_Zymo_D6331": "HiFi_Zymo_D6331.theoretical-distrib.species.csv",
                     "ATCC_MSA1003": "ATCC_MSA1003.theoretical-distrib.species.csv"}

genus_filenames = {"Zymo_D6300": "Zymo_D6300.theoretical-distrib.genus.csv",
                     "HiFi_Zymo_D6331": "HiFi_Zymo_D6331.theoretical-distrib.genus.csv",
                     "ATCC_MSA1003": "ATCC_MSA1003.theoretical-distrib.genus.csv"}

theoretical_distributions_dict = {"species": species_filenames, "genus": genus_filenames}

## Make dict of all kreport files

In [5]:
# filenames for all sourmash kreports

df_thresh_folder = "sourmash-kreports-2022-10-03.genbank"
zero_thresh_folder = "../output.lr-gather/gather"

kreport_dict = defaultdict(dict)

for kreport_folder in [df_thresh_folder]:#, zero_thresh_folder]:
    for ksize in [31,51]:
        for ds, ds_list in all_datasets.items():
            for d in ds_list:
                fname = f"Sourmash-k{ksize}"
                kreportF = f"{kreport_folder}/{d}.dna-k{ksize}-sc1000.gather.genbank.kreport.txt"
                kreport_dict[d].update({fname: kreportF})
for kreport_folder in [zero_thresh_folder]:
    for ksize in [31,51]:
        for ds, ds_list in all_datasets.items():
            for d in ds_list:
                fname = f"Sourmash-k{ksize}_zeroThresh"
                kreportF = f"{kreport_folder}/{d}.dna-k{ksize}-sc1000.gather.genbank.kreport.txt"
                kreport_dict[d].update({fname: kreportF})

In [6]:
# now add non-sourmash kreports
import glob, os
full_kreport_files = glob.glob("All-kreports/*/inputs/*.kreport.txt")
print(len(full_kreport_files))
for f in full_kreport_files:
    if "D6331" in f:
        #mc = "HiFi_Zymo_D6331"
        rd = "HiFi_Zymo_D6331"
    elif "ATCC" in f:
        #mc = "ATCC_MSA1003"
        if "Illumina" in f:
            rd = "Illumina_ATCC_MSA1003"
        else:
            rd = "HiFi_ATCC_MSA1003_full"
    else:
        
       # mc = "Zymo_D6300"
        if "R10" in f:
            rd = "ONT_R10_Zymo_D6300"
        elif "Illumina" in f:
            rd = "Illumina_Zymo_D6300"
        else:
            rd = 'ONT_Q20_Zymo_D6300_full'
    tool = os.path.basename(f).split('.')[0]
    if "Sourmash" in tool: # no need to add these twice
        continue
    kreport_dict[rd].update({tool:f})
# ADD kreports for SIMULATED SHORT  READS 
sim_kreport_files = glob.glob("All-kreports/simread-kreports/*.kreport.txt")
print(len(sim_kreport_files))
for f in sim_kreport_files:
    if "ATCC" in f:
        rd = "HiFi_ATCC_MSA1003_short"
    elif "D6300" in f:
        rd = 'ONT_Q20_Zymo_D6300_short'
    tool = os.path.basename(f).split('.')[0]
    if "Sourmash" in tool: # no need to add these twice
        continue
    kreport_dict[rd].update({tool:f})

72
14


## Parse kreports for each dataset

In [7]:
TaxResult = namedtuple('TaxResult', "taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count")

In [8]:
def assess_read_dataset(dataset_name, read_dataset, theoretical_dict, kreport_dict, namediff_dict):
    taxresults = []
    #sp_count_dict, gn_count_dict = defaultdict(list), defaultdict(list)
    #species_fp_counts, genus_fp_counts = {}, {}
    #fp_counts= {"species": {}, "genus": {}}
    all_toolnames = kreport_dict.keys()
    
    # get theoretical distribution info
    theoretical_species = pd.read_csv(theoretical_dict["species"][dataset_name])
    theoretical_genera = pd.read_csv(theoretical_dict["genus"][dataset_name])
    
    #theoretical_sp = theoretical_species['Taxon'].tolist()
    #theoretical_gn = theoretical_genera['Taxon'].tolist()
    theoretical = {"species": theoretical_species, "genus": theoretical_genera}

    # set minimum detection threshold for this read dataset
    #detect_dict = {'0.001-perc': int(0.00001*total_reads),
    #           '0.1-perc': int(0.001*total_reads),
    #           '1-perc': int(0.01*total_reads)}

    # change this to generate results for other filtering threshold
   # dlevel = '0.001-perc'

    #for k, v in detect_dict.items():
    #    print(k, '=', v)
    detection_thresholds = [0.01, 0.001, 0.0001, 0]
    rank_labels = {"species": 'S', "genus": 'G'}
    for kreport_fname, kreportF in kreport_dict.items():    
        # read kreport
        kreport_df = pd.read_csv(kreportF, sep = '\t', header=None,
                         names = ['proportion', 'cumulative_count', 'level_count', 'rank', 'taxid', 'name'])    
        # First, search for True Positives and False Negatives
        for rank in ["species", "genus"]:
            theoretical_info = theoretical[rank]
            theoretical_taxa = theoretical_info['Taxon'].tolist()
            searchnames = []
            rk = rank_labels[rank]
            for t in theoretical_taxa:
                # for threshold reads /percent, grab total expected reads/percent
                reads_column = f"{read_dataset}_theoretical_reads"
                theoretical_reads = theoretical_info[theoretical_info['Taxon'] == t][reads_column].values[0]
                searchname = namediff_dict.get(t, t)
                searchnames.append(searchname)
                if "|" in searchname:
                    tempD = kreport_df[(kreport_df['name'].str.contains(searchname, regex=True))]
                else:
                    tempD = kreport_df[(kreport_df['name'].str.contains(t))]
                # add the cumulative count for the taxon
                # for species, this will have the effect of including all strains below, which are correct
                this_count = tempD.loc[kreport_df['rank'] == rk, 'cumulative_count'].sum()
               
                detection_info = []
                # for sourmash, check if % exceeds minimum % detection threshold
                if "Sourmash" in kreport_fname: 
                    theoretical_percent = theoretical_info[theoretical_info['Taxon'] == t]["Theoretical Distribution"].values[0]
                    this_perc = tempD.loc[kreport_df['rank'] == rk, 'proportion'].sum()
                    for thresh in detection_thresholds:
                        res ="FN"
                        perc_needed = theoretical_percent * thresh
                        #print(perc_needed)
                        if this_perc >= perc_needed and this_perc !=0:
                            res="TP"
                        detection_info.append(res)
                    #print(detection_info)
                # for all others, check if read count exceeds minimum read threshold
                else:
                    for thresh in detection_thresholds:
                        res ="FN"
                        reads_needed = theoretical_reads * thresh
                        #print(this_count, reads_needed)
                        if this_count >= reads_needed and this_count !=0:
                            res="TP"
                        #print(res)
                        detection_info.append(res)
                taxreslist = [t, rank, dataset_name, read_dataset, kreport_fname] + detection_info + [this_count]
                #print(taxreslist)
                taxres = TaxResult(*taxreslist)
                #print(taxres)
                #taxres = TaxResult(t, rank, dataset_name, read_dataset, kreport_fname, *detection_info, this_count)
                #sp_count_dict[kreport_fname].append(this_count)
                taxresults.append(taxres)
        
            # Now, look for False Positives
            fp_df= kreport_df[(~kreport_df['name'].str.contains("|".join(searchnames), regex=True))]
            rank_fp_df =  fp_df.loc[fp_df['rank'] == rk]
            for taxon in rank_fp_df['name'].to_list():
                this_taxon_count = rank_fp_df[rank_fp_df["name"] == taxon]['cumulative_count'].sum()
                taxres = TaxResult(taxon, rank, dataset_name, read_dataset, kreport_fname, "FP","FP","FP","FP", this_count)
                taxresults.append(taxres)
       
    return taxresults

                 # check if the readcounts at species level exceed the min threshold of reads
                #if temp.loc[df['rank'] == 'S', 'cumulative_count'].sum() >= detect_dict[dlevel]:
                    # if yes, add to true positive count
                #sp_tp_count += 1
                #elif temp.loc[df['rank'] == 'S', 'cumulative_count'].sum() < detect_dict[dlevel]:
                # if no, add to false negative count
                #sp_fn_count += 1
                #if this_count == 0:
                #    result_type="FN"
                    
                # for sourmash, check proportion instead?
                # if "Sourmash" in kreport_fname:
                    #
                    
                        #if "Sourmash" in kreport_fname: 
                #    this_perc =  rank_fp_df[rank_fp_df['rank'] == rk, 'proportion'].sum()
                #    for thresh in detection_thresholds:
                #        res ="FN"
                #        perc_needed = theoretical_percent * thresh
                #        #print(perc_needed)
                #        if this_perc >= perc_needed:
                ##            res="TP"
                #        detection_info.append(res)     
                # for all others, check if read count exceeds minimum read threshold
                #else:
                #    for thresh in detection_thresholds:
                #        res ="FN"
                #        reads_needed = theoretical_reads * thresh
                #        if this_count >= reads_needed:
                #            res="TP"
                #        detection_info.append(res)

### Parse all kreports

In [9]:
#all_fp = defaultdict(dict)
all_taxres = []
for mock_community, read_datasets in all_datasets.items():
    for rd in read_datasets: 
        taxrec = assess_read_dataset(mock_community, rd, theoretical_distributions_dict, kreport_dict[rd], taxa_searchterms)
        all_taxres +=taxrec

### Convert TaxResult list to DataFrame

In [10]:
all_results = pd.DataFrame.from_records(all_taxres, columns=TaxResult._fields)
all_results.head()

,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
0,Escherichia coli,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,TP,TP,TP,TP,4930014000
1,Porphyromonas gingivalis,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,TP,TP,TP,2199344000
2,Luteovulum sphaeroides,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,TP,TP,TP,TP,5550780000
3,Staphylococcus epidermidis,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,TP,TP,TP,347065000
4,Streptococcus mutans,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,TP,TP,TP,1845396000


## Look at False Negatives

In [11]:
all_results[(all_results["dany"] == "FN")]

,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
8,Staphylococcus aureus,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,1237000
10,Acinetobacter baumannii,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,52489999
11,Cutibacterium acnes,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,39940999
12,Helicobacter pylori,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,14347000
13,Lactobacillus gasseri,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,10059000
...,...,...,...,...,...,...,...,...,...,...
48484,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-ONT,FN,FN,FN,FN,0
48486,Saccharomyces cerevisiae,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-ONT,FN,FN,FN,FN,0
48500,Salmonella,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-ONT,FN,FN,FN,FN,0
48501,Enterococcus,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-ONT,FN,FN,FN,FN,0


In [12]:
fn_D6331 = all_results[(all_results["dany"] == "FN")]# & (all_results["read_dataset"] == "HiFi_Zymo_D6331")]
fn_D6331

,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
8,Staphylococcus aureus,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,1237000
10,Acinetobacter baumannii,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,52489999
11,Cutibacterium acnes,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,39940999
12,Helicobacter pylori,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,14347000
13,Lactobacillus gasseri,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,10059000
...,...,...,...,...,...,...,...,...,...,...
48484,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-ONT,FN,FN,FN,FN,0
48486,Saccharomyces cerevisiae,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-ONT,FN,FN,FN,FN,0
48500,Salmonella,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-ONT,FN,FN,FN,FN,0
48501,Enterococcus,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-ONT,FN,FN,FN,FN,0


In [13]:
fn_D6331['tool'].nunique()

16

In [14]:
fn_D6331.groupby(["taxon", "rank"])['cumulative_count'].value_counts().reset_index(name = 'count')#.as_index().reindex()

,taxon,rank,cumulative_count,count
0,Acinetobacter,genus,2059000,2
1,Acinetobacter,genus,6735000,2
2,Acinetobacter,genus,52489999,2
3,Acinetobacter,genus,1586000,1
4,Acinetobacter,genus,1601999,1
...,...,...,...,...
245,Staphylococcus aureus,species,679000,1
246,Staphylococcus aureus,species,932000,1
247,Staphylococcus aureus,species,5636999,1
248,Streptococcus agalactiae,species,0,2


In [16]:
fn_sp = all_results[(all_results["dany"] == "FN") & (all_results["rank"] == "species")]
fn_gn = all_results[(all_results["dany"] == "FN") & (all_results["rank"] == "genus")]
fn_gn

,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
33,Acinetobacter,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,52489999
34,Cutibacterium,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,39940999
35,Helicobacter,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,14347000
36,Lactobacillus,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,10059000
37,Neisseria,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,29623000
...,...,...,...,...,...,...,...,...,...,...
46041,Saccharomyces,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Centrifuge-h500,FN,FN,FN,FN,0
47856,Saccharomyces,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Centrifuge-h22,FN,FN,FN,FN,0
48500,Salmonella,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-ONT,FN,FN,FN,FN,0
48501,Enterococcus,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-ONT,FN,FN,FN,FN,0


In [17]:
fn_gn.groupby(["read_dataset", "tool"])['cumulative_count'].value_counts().reset_index(name = 'count')#.as_index().reindex()

,read_dataset,tool,cumulative_count,count
0,HiFi_ATCC_MSA1003_full,BugSeq-V2,0,3
1,HiFi_ATCC_MSA1003_full,MEGAN-LR-Nuc-HiFi,0,5
2,HiFi_ATCC_MSA1003_full,MEGAN-LR-Nuc-ONT,0,5
3,HiFi_ATCC_MSA1003_full,MEGAN-LR-Prot,0,4
4,HiFi_ATCC_MSA1003_full,Metaphlan3,0,3
...,...,...,...,...
174,ONT_R10_Zymo_D6300,Sourmash-k51,12079000,1
175,ONT_R10_Zymo_D6300,Sourmash-k51,14583000,1
176,ONT_R10_Zymo_D6300,Sourmash-k51_zeroThresh,12079000,1
177,ONT_R10_Zymo_D6300,Sourmash-k51_zeroThresh,14583000,1


In [18]:
fn_sp.groupby(["tool", "read_dataset"])['cumulative_count'].value_counts().reset_index(name = 'count')#.as_index().reindex()

,tool,read_dataset,cumulative_count,count
0,Bracken,HiFi_Zymo_D6331,0,3
1,BugSeq-V2,HiFi_ATCC_MSA1003_full,0,3
2,BugSeq-V2,HiFi_Zymo_D6331,0,5
3,Centrifuge-h22,HiFi_Zymo_D6331,0,4
4,Centrifuge-h22,Illumina_Zymo_D6300,0,2
...,...,...,...,...
201,mOTUs,HiFi_Zymo_D6331,0,5
202,mOTUs,Illumina_ATCC_MSA1003,0,5
203,mOTUs,Illumina_Zymo_D6300,0,2
204,mOTUs,ONT_Q20_Zymo_D6300_full,0,2


In [19]:
all_results[(all_results["taxon"] == "Clostridium") & (all_results["read_dataset"] == "HiFi_Zymo_D6331")]# & (all_results["result_type"] == "FN")]

,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
43784,Clostridium,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Sourmash-k31,FN,FN,FN,FN,0
43821,Clostridium,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Sourmash-k51,FN,FN,FN,FN,0
43858,Clostridium,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Sourmash-k31_zeroThresh,FN,FN,FN,FN,0
44187,Clostridium,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Sourmash-k51_zeroThresh,FN,FN,FN,FN,11222999
44512,Clostridium,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Metamaps,TP,TP,TP,TP,44
44768,Clostridium,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MMseqs2,TP,TP,TP,TP,703
44849,Clostridium,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-HiFi,FN,FN,FN,FN,0
44885,Clostridium,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Prot,TP,TP,TP,TP,381
45501,Clostridium,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Kraken2,TP,TP,TP,TP,309
45847,Clostridium,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,BugSeq-V2,FN,FN,FN,FN,0


In [20]:
fn_sp[(fn_sp["tool"] == "Sourmash-k31")]

,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
8,Staphylococcus aureus,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,1237000
10,Acinetobacter baumannii,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,52489999
11,Cutibacterium acnes,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,39940999
12,Helicobacter pylori,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,14347000
13,Lactobacillus gasseri,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,10059000
14,Neisseria meningitidis,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,29623000
15,Phocaeicola vulgatus,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,16535000
16,Bifidobacterium adolescentis,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,481000
17,Deinococcus radiodurans,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,4126000
18,Enterococcus faecalis,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FN,FN,FN,FN,273000


In [21]:
all_results[(all_results["taxon"] == "Clostridium perfringens") & (all_results["read_dataset"] == "HiFi_Zymo_D6331")]#["result_type"].value_counts()#.reset_index(name = 'result_count')#.value_counts()# & (all_results["result_type"] == "FN")]

,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
43765,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Sourmash-k31,FN,FN,FN,FN,0
43801,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Sourmash-k51,FN,FN,FN,FN,0
43839,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Sourmash-k31_zeroThresh,FN,FN,FN,FN,0
43875,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Sourmash-k51_zeroThresh,FN,FN,FN,FN,0
44334,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Metamaps,FN,FN,FN,FN,0
44637,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MMseqs2,FN,FN,FN,FN,0
44830,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Nuc-HiFi,FN,FN,FN,FN,0
44867,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,MEGAN-LR-Prot,FN,FN,FN,FN,0
44902,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Kraken2,TP,TP,TP,TP,3
45830,Clostridium perfringens,species,HiFi_Zymo_D6331,HiFi_Zymo_D6331,BugSeq-V2,FN,FN,FN,FN,0


## Look at False Positives

In [30]:
fp_sp = all_results[(all_results["dany"] == "FP") & (all_results["rank"] == "species")]
fp_gn = all_results[(all_results["dany"] == "FP") & (all_results["rank"] == "genus")]
fp_gn

,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
43,Bradyrhizobium,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FP,FP,FP,FP,654000
44,Paulinella,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FP,FP,FP,FP,654000
86,Bradyrhizobium,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k51,FP,FP,FP,FP,637000
130,Bradyrhizobium,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31_zeroThresh,FP,FP,FP,FP,654000
131,Paulinella,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31_zeroThresh,FP,FP,FP,FP,654000
...,...,...,...,...,...,...,...,...,...,...
48466,Mimivirus,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Centrifuge-h22,FP,FP,FP,FP,0
48467,Chloriridovirus,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Centrifuge-h22,FP,FP,FP,FP,0
48468,Oryzopoxvirus,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Centrifuge-h22,FP,FP,FP,FP,0
48469,Pomovirus,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Centrifuge-h22,FP,FP,FP,FP,0


In [31]:
fp_sp_counts = fp_sp.groupby(["tool", "read_dataset"])['cumulative_count'].value_counts().reset_index(name = 'count')#.as_index().reindex()
fp_sp_counts

,tool,read_dataset,cumulative_count,count
0,Bracken,HiFi_ATCC_MSA1003_full,142,143
1,Bracken,HiFi_ATCC_MSA1003_short,1310,422
2,Bracken,HiFi_Zymo_D6331,3987,242
3,Bracken,Illumina_ATCC_MSA1003,1055,455
4,Bracken,Illumina_Zymo_D6300,81091,439
...,...,...,...,...
80,Sourmash-k51_zeroThresh,ONT_R10_Zymo_D6300,12079000,3
81,mOTUs,HiFi_ATCC_MSA1003_full,0,1
82,mOTUs,Illumina_ATCC_MSA1003,0,1
83,mOTUs,Illumina_Zymo_D6300,0,2


In [32]:
fp_sp[(fp_sp["tool"] == "Sourmash-k31")]

,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
20,Staphylococcus haemolyticus,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FP,FP,FP,FP,654000
21,Bradyrhizobium viridifuturi,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FP,FP,FP,FP,654000
22,Bacillus pacificus,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FP,FP,FP,FP,654000
23,Paulinella micropora,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FP,FP,FP,FP,654000
24,Rhodobacter sp. AKP1,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FP,FP,FP,FP,654000
2174,Staphylococcus haemolyticus,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_short,Sourmash-k31,FP,FP,FP,FP,155000
2175,Bradyrhizobium sp. BL,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_short,Sourmash-k31,FP,FP,FP,FP,155000
2176,Rhodobacter sp. AKP1,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_short,Sourmash-k31,FP,FP,FP,FP,155000
7088,Staphylococcus haemolyticus,species,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k31,FP,FP,FP,FP,100000
7089,Coremiostelium polycephalum,species,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k31,FP,FP,FP,FP,100000


In [78]:
#fp_sp[(fp_sp["mock_community"] == "ATCC_MSA1003") & (fp_sp['taxon'] == "Staphylococcus haemolyticus")]
#fp_sp[(fp_sp["mock_community"] == "ATCC_MSA1003") & (fp_sp['taxon'] == "Bradyrhizobium viridifuturi")]
#fp_sp[(fp_sp["mock_community"] == "ATCC_MSA1003") & (fp_sp['taxon'] == "Bacillus pacificus")]
#fp_sp[(fp_sp["mock_community"] == "ATCC_MSA1003") & (fp_sp['taxon'] == "Paulinella micropora")]
fp_sp[(fp_sp["mock_community"] == "ATCC_MSA1003") & (fp_sp['taxon'] == "Rhodobacter sp. AKP1")]


,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
24,Rhodobacter sp. AKP1,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FP,FP,FP,FP,654000
111,Rhodobacter sp. AKP1,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31_zeroThresh,FP,FP,FP,FP,654000
165,Rhodobacter sp. AKP1,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k51_zeroThresh,FP,FP,FP,FP,637000
2176,Rhodobacter sp. AKP1,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_short,Sourmash-k31,FP,FP,FP,FP,155000
2259,Rhodobacter sp. AKP1,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_short,Sourmash-k31_zeroThresh,FP,FP,FP,FP,155000
2309,Rhodobacter sp. AKP1,species,ATCC_MSA1003,HiFi_ATCC_MSA1003_short,Sourmash-k51_zeroThresh,FP,FP,FP,FP,141000
7094,Rhodobacter sp. AKP1,species,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k31,FP,FP,FP,FP,100000
7189,Rhodobacter sp. AKP1,species,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k31_zeroThresh,FP,FP,FP,FP,100000
7244,Rhodobacter sp. AKP1,species,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k51_zeroThresh,FP,FP,FP,FP,77000


In [61]:
lr_tools

msa1003_fp = fp_sp[(fp_sp["mock_community"] == "ATCC_MSA1003") & fp_sp["tool"].isin(comparable_lr_tools)].drop(columns = ["d1perc","dp1perc", "dp01perc","dany", "rank"])#.count().reset_index()#.groupby("taxon").reset_index()
msa1003_fp

,taxon,mock_community,read_dataset,tool,cumulative_count
1212,Luteovulum johrii,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,MMseqs2,67
1213,Rhodobacter sp. AKP1,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,MMseqs2,67
1214,uncultured Rubellimicrobium sp.,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,MMseqs2,67
1215,Rubellimicrobium thermophilum,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,MMseqs2,67
1216,Roseinatronobacter thiooxidans,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,MMseqs2,67
...,...,...,...,...,...
1290,Cy...,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,MMseqs2,67
1291,...,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,MMseqs2,67
1292,...,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,MMseqs2,67
1293,Staphylococcus phage phiSa...,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,MMseqs2,67


In [72]:
msa1003_fp.groupby(["taxon"])['tool'].count().reset_index().sort_values(by="tool", ascending=False)

,taxon,tool
0,...,1
62,uncultured Rubellimicrobium sp.,1
60,Streptococcus troglodytae,1
59,Streptococcus pneumoniae,1
58,Streptococcus peroris,1
...,...,...
26,Chlamydia trachomatis,1
25,Bacillus wiedmannii,1
24,Bacillus thuringiensis,1
23,Bacillus sp. M21,1


In [33]:
fp_gn_counts = fp_gn.groupby(["read_dataset", "tool"])['cumulative_count'].value_counts().reset_index(name = 'count')#.as_index().reindex()
fp_gn_counts

,read_dataset,tool,cumulative_count,count
0,HiFi_ATCC_MSA1003_full,Bracken,142,30
1,HiFi_ATCC_MSA1003_full,Centrifuge-h22,142,34
2,HiFi_ATCC_MSA1003_full,Centrifuge-h500,22,14
3,HiFi_ATCC_MSA1003_full,Kraken2,142,78
4,HiFi_ATCC_MSA1003_full,MMseqs2,68,42
...,...,...,...,...
65,ONT_R10_Zymo_D6300,Metaphlan3,617,10
66,ONT_R10_Zymo_D6300,Sourmash-k31,19694000,2
67,ONT_R10_Zymo_D6300,Sourmash-k31_zeroThresh,19694000,2
68,ONT_R10_Zymo_D6300,Sourmash-k51,12079000,1


In [34]:
fp_gn[(fp_gn["tool"] == "Sourmash-k51")]

,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
86,Bradyrhizobium,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k51,FP,FP,FP,FP,637000
2236,Bradyrhizobium,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_short,Sourmash-k51,FP,FP,FP,FP,141000
7160,Acytostelium,genus,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k51,FP,FP,FP,FP,77000
7161,Saccharomyces,genus,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k51,FP,FP,FP,FP,77000
7162,Phyllobacterium,genus,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k51,FP,FP,FP,FP,77000
22597,Paulinella,genus,Zymo_D6300,ONT_Q20_Zymo_D6300_full,Sourmash-k51,FP,FP,FP,FP,61960000
42559,Streptococcus,genus,Zymo_D6300,ONT_R10_Zymo_D6300,Sourmash-k51,FP,FP,FP,FP,12079000
43824,Acinetobacter,genus,HiFi_Zymo_D6331,HiFi_Zymo_D6331,Sourmash-k51,FP,FP,FP,FP,38605999


In [35]:
fp_gn[(fp_gn["tool"] == "Sourmash-k31")]

,taxon,rank,mock_community,read_dataset,tool,d1perc,dp1perc,dp01perc,dany,cumulative_count
43,Bradyrhizobium,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FP,FP,FP,FP,654000
44,Paulinella,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_full,Sourmash-k31,FP,FP,FP,FP,654000
2195,Bradyrhizobium,genus,ATCC_MSA1003,HiFi_ATCC_MSA1003_short,Sourmash-k31,FP,FP,FP,FP,155000
7113,Coremiostelium,genus,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k31,FP,FP,FP,FP,100000
7114,Saccharomyces,genus,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k31,FP,FP,FP,FP,100000
7115,Phyllobacterium,genus,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k31,FP,FP,FP,FP,100000
7116,Pterula,genus,ATCC_MSA1003,Illumina_ATCC_MSA1003,Sourmash-k31,FP,FP,FP,FP,100000
22572,Paulinella,genus,Zymo_D6300,ONT_Q20_Zymo_D6300_full,Sourmash-k31,FP,FP,FP,FP,74842000
27878,Paulinella,genus,Zymo_D6300,ONT_Q20_Zymo_D6300_short,Sourmash-k31,FP,FP,FP,FP,47457000
42534,Paulinella,genus,Zymo_D6300,ONT_R10_Zymo_D6300,Sourmash-k31,FP,FP,FP,FP,19694000


In [36]:
fp_gn_counts[fp_gn_counts["read_dataset"] == "HiFi_Zymo_D6331"]

,read_dataset,tool,cumulative_count,count
18,HiFi_Zymo_D6331,Bracken,3987,148
19,HiFi_Zymo_D6331,Centrifuge-h22,0,613
20,HiFi_Zymo_D6331,Centrifuge-h500,0,26
21,HiFi_Zymo_D6331,Kraken2,3993,312
22,HiFi_Zymo_D6331,MEGAN-LR-Nuc-HiFi,3819,1
23,HiFi_Zymo_D6331,MEGAN-LR-Nuc-ONT,3874,1
24,HiFi_Zymo_D6331,MMseqs2,3401,45
25,HiFi_Zymo_D6331,Metamaps,3996,108
26,HiFi_Zymo_D6331,Metaphlan3,653,19
27,HiFi_Zymo_D6331,Sourmash-k51,38605999,1


In [37]:
fp_sp_counts[fp_sp_counts["read_dataset"] == "HiFi_Zymo_D6331"]

,tool,read_dataset,cumulative_count,count
2,Bracken,HiFi_Zymo_D6331,3987,242
12,Centrifuge-h22,HiFi_Zymo_D6331,0,1331
19,Centrifuge-h500,HiFi_Zymo_D6331,0,45
24,Kraken2,HiFi_Zymo_D6331,3987,582
30,MEGAN-LR-Nuc-HiFi,HiFi_Zymo_D6331,0,2
31,MEGAN-LR-Nuc-ONT,HiFi_Zymo_D6331,0,1
32,MEGAN-LR-Prot,HiFi_Zymo_D6331,2196,1
34,MMseqs2,HiFi_Zymo_D6331,810,114
38,Metamaps,HiFi_Zymo_D6331,3996,161
43,Metaphlan3,HiFi_Zymo_D6331,636,26
